# 13-json输出解析器

此输出解析器允许用户指定任意 JSON 架构并查询 LLMs 以获得符合该架构的输出。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 JSON。在OpenAI家族中，达芬奇可以可靠地完成任务，但居里的能力已经急剧下降。

您可以选择使用 Pydantic 来声明您的数据模型。

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)


In [4]:
# 自定义输出的数据结构
class Book(BaseModel):
    title: str = Field(description="书名")
    author: str = Field(description="作者")
    year: int = Field(description="出版年份")
    price: float = Field(description="价格")
    description: str = Field(description="书的简介")

# 创建输出解析器
output_parser = JsonOutputParser(pydantic_object=Book)

format_instructions = output_parser.get_format_instructions()
format_instructions = """输出符合以下格式的JSON字符串实例: \n
```JSON
{
    "title": "书名",
    "author": "作者",
    "year": "出版年份",
    "price": "价格",
    "description": "书的简介"
}
```
"""

# 定义提示词模板
prompt = PromptTemplate(
    template="{format_instructions}\n请列出{query}的信息",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

# 创建链
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"query": "如何开始学习中国历史"})

{'title': '中国通史简明教程',
 'author': '张三',
 'year': '2023',
 'price': '59.99元',
 'description': '本书是一本适合初学者阅读的中国历史入门书籍，从远古时期到现代，全面介绍了中国的政治、经济和社会变迁。'}

In [10]:
chain.invoke({"query": "红楼梦"})

{'title': '红楼梦',
 'author': '曹雪芹',
 'year': '1791',
 'price': '35.00元',
 'description': '《红楼梦》，中国古代四大名著之一，章回体长篇小说，成书于1791年。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了一批举止见识出于须眉之上的闺阁佳人的人生百态，展现了真正的人性美和悲剧美，可以说是一部从各个角度展现女性美的史诗。'}